# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
from citipy import citipy

output_data_file = "../output_data/Vacation.csv"


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
airport_df = pd.read_csv("../output_data/cities.csv")

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = airport_df[["Latitude", "Longitude"]]
humidity = airport_df["Humidity (%)"].astype(float)
humidity_percent = humidity/100

In [4]:
figure_layout = {
    'width': '1000px',
    'eight': '600px',
    'order': '1px solid black',
    'adding': '1px'}
fig = gmaps.figure(center=[25,10], zoom_level=2,layout=figure_layout)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity)

fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px', order='1px solid black', width='1000px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
hotel_df = airport_df[airport_df['Temperature (F)'] < 80]
hotel_df = airport_df[airport_df['Temperature (F)'] > 70]
hotel_df = airport_df[airport_df['Wind Speed (mph)'] < 10]
hotel_df = airport_df[airport_df['Cloudiness (%)'] < 0.001]

hotel_df.to_csv(output_data_file, index=False, header=True)
hotel_df

,City,Country,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
1,Kuito,AO,-12.38,16.93,57.61,43,0,3.02
11,Ubari,LY,26.59,12.78,80.11,21,0,6.89
15,Yar-Sale,RU,66.83,70.83,54.61,73,0,11.88
24,Albany,US,42.60,-73.97,87.46,49,0,3.60
41,Busselton,AU,-33.65,115.33,34.97,87,0,7.00
...,...,...,...,...,...,...,...,...
509,Sekoma,BW,-24.40,23.88,54.39,50,0,7.58
526,Inhambane,MZ,-23.86,35.38,67.87,79,0,6.42
529,Erenhot,CN,43.65,111.98,71.51,43,0,7.38
532,La Providencia,MX,28.72,-111.58,96.04,46,0,15.99


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Add a "Hotel Name" column to the DataFrame.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
hotel_df['Hotel Name'] = ""

# target_coordinates = coordinates
# target_search = "City"
# target_radius = 5000
# target_type = "lodging"

for index, row in hotel_df.iterrows():
    coordinates = str(hotel_df.loc[index,'Latitude']) +',' + str(hotel_df.loc[index,'Longitude'])
    params = {
        "location": coordinates,
        "radius": 5000,
        "type": "lodging",
        "key": g_key
        }
    response = requests.get(base_url, params=params).json()
    results = response['results']
    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name'];
    except (IndexError):
        continue

hotel_df.to_csv(output_data_file, index=False, header=True)
hotel_df.head()

# set up a parameters dictionary
# run a request using our params dictionary

C:\Users\Mitch\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Mitch\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,City,Country,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hotel Name
1,Kuito,AO,-12.38,16.93,57.61,43,0,3.02,Herdeiros de Antunes Capusso
11,Ubari,LY,26.59,12.78,80.11,21,0,6.89,فندق أوباري
15,Yar-Sale,RU,66.83,70.83,54.61,73,0,11.88,Yalemd
24,Albany,US,42.60,-73.97,87.46,49,0,3.60,
41,Busselton,AU,-33.65,115.33,34.97,87,0,7.00,Observatory Guest House


In [7]:
# NOTE: Do not change any of the code in this cell
narrowed_city_df = hotel_df
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
hotel_locations = hotel_df[["Latitude", "Longitude"]]

In [8]:
# Add marker layer ontop of heat map
df= pd.read_csv(output_data_file)
locations = df[["Latitude", "Longitude"]]
Humidity = df["Humidity (%)"]

figure_layout = {
    'width': '1000px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px'
}
fig = gmaps.figure(center=[25,10], zoom_level=2,layout=figure_layout)
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity)
fig.add_layer(heat_layer)

markers = gmaps.marker_layer(hotel_locations,info_box_content=hotel_info)
fig.add_layer(markers)

fig

# Display Map

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='1000px'))